### Instalação das dependencias


In [ ]:
!pip install feedparser beautifulsoup4
!pip install transformers sentence-transformers
!pip install bitsandbytes accelerate
!pip install faiss-cpu -q

In [ ]:
import re
import gc
import numpy as np
import pandas as pd
import torch
import faiss
import feedparser
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

### Definição das Fontes RSS

In [ ]:
RSS_FEEDS = [
    # --- Notícias Locais (Paraíba) ---
    "https://www.jornaldaparaiba.com.br/rss",
    "https://www.clickpb.com.br/rss",
    "https://www.paraibaonline.com.br/rss",
    "https://www.pbagora.com.br/rss",
    "https://f5online.com.br/feed/",
    "https://www.portalt5.com.br/noticias/feed.rss",
    "https://www.polemicaparaiba.com.br/feed/",
    "https://www.maispb.com.br/feed",                 # MaisPB
    "https://www.wscom.com.br/feed/",                 # WSCOM Notícias

    # --- Regionais (Nordeste) ---
    "https://www.diariodonordeste.com.br/rss.xml",    # Diário do Nordeste (CE)
    "https://www.op9.com.br/feed/",                   # OP9 (Nordeste)
    "https://www.cbnrecife.com/rss",                  # CBN Recife
    "https://www.jornaldocomercio.com/_conteudo/feed",# JC Online (PE)
    "https://www.atarde.com.br/rss",                  # A Tarde (BA)

    # --- Nacionais (Portais Diversos) ---
    "http://rss.uol.com.br/feed/noticias.xml",        # UOL Notícias
    "https://noticias.r7.com/feed.xml",               # R7 Notícias
    "https://www.estadao.com.br/rss/ultimas.xml",     # Estadão Últimas
    "http://feeds.folha.uol.com.br/emcimadahora/rss091.xml", # Folha em Cima da Hora
    "https://valor.globo.com/rss/",                   # Valor Econômico
    "https://www.infomoney.com.br/feed/",             # InfoMoney
    "https://oglobo.globo.com/rss.xml",               # O Globo
    "https://www.cartacapital.com.br/feed/",          # Carta Capital
    "https://www.conjur.com.br/index.php/feed",       # Consultor Jurídico (Direito/Política)

    # --- Nacionais (G1, por editoria) ---
    "https://g1.globo.com/rss/g1/brasil/",
    "https://g1.globo.com/rss/g1/carros/",
    "https://g1.globo.com/rss/g1/ciencia-e-saude/",
    "https://g1.globo.com/rss/g1/economia/",
    "https://g1.globo.com/rss/g1/educacao/",
    "https://g1.globo.com/rss/g1/mundo/",
    "https://g1.globo.com/rss/g1/politica/",
    "https://g1.globo.com/rss/g1/tecnologia/",
    "https://g1.globo.com/rss/g1/pb/paraiba/",
    "https://g1.globo.com/rss/g1/pop-arte/",           # Cultura
    "https://g1.globo.com/rss/g1/natureza/",           # Meio Ambiente
    "https://g1.globo.com/rss/g1/turismo-e-viagem/",   # Viagem
    "https://g1.globo.com/rss/g1/cultura/",            # Artes e cultura
    "https://g1.globo.com/rss/g1/fato-ou-fake/",       # Checagem de fatos
    # Google News - consultas personalizadas
    "https://news.google.com/rss/search?q=Paraíba&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Brasil&hl=pt-BR&gl=BR&ceid=BR:pt-419",

    # Capitais brasileiras
    "https://news.google.com/rss/search?q=Rio+Branco&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Maceió&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Macapá&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Manaus&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Salvador&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Fortaleza&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Brasília&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Vitória&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Goiânia&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=São+Luís&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Cuiabá&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Campo+Grande&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Belo+Horizonte&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Belém&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=João+Pessoa&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Curitiba&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Recife&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Teresina&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Rio+de+Janeiro&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Natal&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Porto+Alegre&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Porto+Velho&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Boa+Vista&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Florianópolis&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=São+Paulo&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Aracaju&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Palmas&hl=pt-BR&gl=BR&ceid=BR:pt-419"

]


## Coleta e Pré-processamento dos Dados

In [ ]:
def fetch_and_clean_news(feed_urls):
    articles = []
    for url in feed_urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            soup = BeautifulSoup(entry.get('summary', ''), 'html.parser')
            text_content = re.sub(r'\s+', ' ', soup.get_text()).strip()

            articles.append({
                'fonte': feed.feed.get('title', 'N/A'),
                'titulo': entry.title,
                'link': entry.link,
                'descricao': text_content,
                'data': entry.get('published', 'N/A')
            })
    return pd.DataFrame(articles)

df_feeds = fetch_and_clean_news(RSS_FEEDS)


In [ ]:
print(f"Total de notícias coletadas: {len(df_feeds)}")
df_feeds.head()

Total de notícias coletadas: 4721


,fonte,titulo,link,descricao,data
0,Jornal da Paraíba,Atualização do Pix disponibilza 'botão de cont...,https://jornaldaparaiba.com.br/tecnologia/atua...,Atualização do Pix disponibilza 'botão de cont...,"Tue, 30 Sep 2025 17:34:39 GMT"
1,Jornal da Paraíba,"Com crise financeira, UEPB foi quase 'esquecid...",https://jornaldaparaiba.com.br/politica/pleno-...,Assembleia autoriza governo a contratar emprés...,"Tue, 30 Sep 2025 17:30:17 GMT"
2,Jornal da Paraíba,Quem é Fatoka: chefe do Comando Vermelho fugiu...,https://jornaldaparaiba.com.br/cotidiano/quem-...,"Quem é Fatoka, chefe do Comando Vermelho na Pa...","Tue, 30 Sep 2025 17:17:24 GMT"
3,Jornal da Paraíba,STF forma maioria a favor de decisão de Fux qu...,https://jornaldaparaiba.com.br/politica/conver...,Supremo Tribunal Federal. (Foto: Divulgação/Co...,"Tue, 30 Sep 2025 17:04:45 GMT"
4,Jornal da Paraíba,Confira a artilharia da 2ª divisão do Campeona...,https://jornaldaparaiba.com.br/esportes/confir...,A disputas da 2ª divisão do Campeonato Paraiba...,"Tue, 30 Sep 2025 15:51:15 GMT"


In [ ]:
df_feeds['content_for_embedding'] = df_feeds['titulo'] + ". " + df_feeds['descricao']

Dados salvos em feeds-2025-09-30-processado.csv


## Vetorização do Conteúdo e Indexação

In [ ]:
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cuda')

contents_to_embed = df_feeds['content_for_embedding'].tolist()
embeddings = embedding_model.encode(contents_to_embed, show_progress_bar=True)

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings).astype('float32'))

print(f"\nShape dos embeddings: {embeddings.shape}")
print(f"Total de documentos indexados no FAISS: {index.ntotal}")

/home/leandrojackson/anaconda3/envs/arte/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|████████████████████████████████| 148/148 [00:02<00:00, 71.65it/s]


Shape dos embeddings: (4721, 384)
Total de documentos indexados no FAISS: 4721


In [ ]:
def search_similar_news(query, k=4):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)
    retrieved_docs = df_feeds.iloc[indices[0]]
    return retrieved_docs

## Configuração do Modelo de Linguagem (LLM) para Geração

In [ ]:
#model_name = "stabilityai/stablelm-zephyr-3b"
model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_compute_dtype": torch.bfloat16,
}

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    trust_remote_code=True,
)


tokenizer = AutoTokenizer.from_pretrained(model_name)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
)

Loading checkpoint shards: 100%|██████████████████| 8/8 [00:07<00:00,  1.14it/s]
Device set to use cuda:0


## 	Sistema de Resposta com Geração Aumentada por Recuperação (RAG)

In [ ]:
def generate_answer(query, search_results):
    descriptions = [d[:2000] for d in search_results['descricao'].tolist()]
    context = "\n\n".join(descriptions)

    prompt_template = f"""<|system|>
Você é um assistente de notícias especializado em análise precisa e linguagem sofisticada. Sua tarefa é responder à pergunta do usuário usando APENAS o contexto fornecido. Não invente informações além do contexto. Se a resposta não estiver no contexto, diga: "Com base nas notícias analisadas, não tenho informações suficientes para responder a essa pergunta."

Regras:
1. Use linguagem formal e, quando possível, termos técnicos ou eruditos relacionados ao tema.
2. Seja conciso, mas inclua análise breve quando existirem dados conflitantes.
3. Se houver dados conflitantes, explique brevemente a divergência antes de responder.
4. Sempre indique claramente se a informação é insuficiente para responder.
5. Resuma em 2 ou 3 frases claras e objetivas.
6. Não use linguagem coloquial.

Contexto:
{context}

Pergunta: {query}</s>
<|assistant|>
"""

    response = text_generator(prompt_template)

    return response[0]['generated_text'].split("<|assistant|>")[1].strip()


### Exemplos

In [ ]:
torch.cuda.empty_cache()
gc.collect()

user_query = "uais as novidades sobre a prefeitura de João Pessoa?"

retrieved_news = search_similar_news(user_query, k=10)

print("--- Notícias recuperadas para gerar a resposta: ---")
display(retrieved_news[['titulo', 'fonte']])
print("--------------------------------------------------\n")

final_answer = generate_answer(user_query, retrieved_news)

print(f"Pergunta do Usuário: {user_query}\n")
print(f"Resposta do Chatbot:\n{final_answer}")

--- Notícias recuperadas para gerar a resposta: ---


,titulo,fonte
1193,Justiça da Paraíba manda desocupar prédio cons...,g1 > Paraíba
1220,"Prefeito e vice de Soledade, na PB, têm mandat...",g1 > Paraíba
1005,Tarcísio foi na onda do Centrão e se deu mal c...,g1 > Política
160,Lula retorna à Bahia na próxima semana; veja d...,A TARDE
652,Tarcísio prioriza visita a Bolsonaro e faltará...,g1 > Economia
964,Tarcísio prioriza visita a Bolsonaro e faltará...,g1 > Política
930,Governo de SP anuncia gabinete de crise e fech...,g1 > Política
714,"Após nova operação contra o PCC, Haddad anunci...",g1 > Economia
123,Banheiros dos quiosques da orla de João Pessoa...,"WSCOM – Quem sabe, faz conteúdo"
973,Tarcísio visita Bolsonaro nesta segunda em Bra...,g1 > Política


--------------------------------------------------



/home/leandrojackson/anaconda3/envs/arte/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/leandrojackson/anaconda3/envs/arte/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Pergunta do Usuário: uais as novidades sobre a prefeitura de João Pessoa?

Resposta do Chatbot:
Uma nova prédica em João Pessoa ordenou a desocupação de um prédio construído acima da altura permitida. O juiz Antônio Carneiro de Paiva Júnior, da 4ª Vara da Fazenda Pública de João Pessoa, determinou a desocupação do prédio em relação à "Habite-se", já que o prédio funciona sem a licença de habitação. A decisão foi tomada na terça-feira (24) e foi recomendada pelo Ministério Público da Paraíba (MPPB). Foi fixado um período de 10 dias para a desocupação do prédio, caso contrário, deverá haver multa diária de R$ 5 mil. A anistia é um dos temas abordados por Tarcísio de Freitas, governador de São Paulo, durante sua visita a Brasília.


In [ ]:

torch.cuda.empty_cache()
gc.collect()

#user_query = "O dolar aumentou ou desceu?"
user_query = "O dólar incidiu em apreciação ou padeceu de depreciação?"
#user_query = "O dólar incorreu em hipertrofia valorativa ou padeceu de depauperação cambial?"
retrieved_news = search_similar_news(user_query, k=5)

print("--- Notícias recuperadas para gerar a resposta: ---")
display(retrieved_news[['titulo', 'fonte']])
print("--------------------------------------------------\n")

final_answer = generate_answer(user_query, retrieved_news)

print(f"Pergunta do Usuário: {user_query}\n")
print(f"Resposta do Chatbot:\n{final_answer}")

--- Notícias recuperadas para gerar a resposta: ---


,titulo,fonte
699,"Dólar cai e fecha a semana a R$ 5,33, após dad...",g1 > Economia
636,"Dólar fecha estável a R$ 5,32 com temor de par...",g1 > Economia
723,"Dólar sobe e fecha a R$ 5,36 após dado forte d...",g1 > Economia
663,"Dólar cai e fecha a R$ 5,32 com maior apetite ...",g1 > Economia
691,BC da Argentina impõe controle cambial para ev...,g1 > Economia


--------------------------------------------------

Pergunta do Usuário: O dólar incidiu em apreciação ou padeceu de depreciação?

Resposta do Chatbot:
O texto fornece informações sobre o desempenho do dólar em diferentes dias, e a resposta à pergunta depende do dia específico que se refere. O dólar fechou em queda de 0,47% em 26 de agosto, em alta de 0,69% em 25 de agosto, em queda de 0,31% em 29 de agosto, e acumulou queda de 13,88% no ano. Portanto, no último dia mencionado, o dólar padeceu de depreciação. No entanto, em outros dias, a apreciação ou a depreciação do dólar variam.


In [ ]:

import torch
import gc

torch.cuda.empty_cache()
gc.collect()

user_query = "O que o prefeito de joão pessoa tem feito?"
#user_query = "O dólar incidiu em apreciação ou padeceu de depreciação?"
#user_query = "O dólar incorreu em hipertrofia valorativa ou padeceu de depauperação cambial?"
retrieved_news = search_similar_news(user_query, k=10)

print("--- Notícias recuperadas para gerar a resposta: ---")
display(retrieved_news[['titulo', 'fonte']])
print("--------------------------------------------------\n")

final_answer = generate_answer(user_query, retrieved_news)

print(f"Pergunta do Usuário: {user_query}\n")
print(f"Resposta do Chatbot:\n{final_answer}")

--- Notícias recuperadas para gerar a resposta: ---


,titulo,fonte
1222,"Prefeito e vice de Soledade, na PB, têm mandat...",g1 > Paraíba
2125,"Saiba quem é o prefeito de Macapá, suspeito de...","""Macapá"" - Google Notícias"
4687,Morte de prefeito completa sete anos sem nenhu...,"""Palmas"" - Google Notícias"
4307,Prefeito de Florianópolis dispara: “cadê os di...,"""Florianópolis"" - Google Notícias"
3549,Política Municipal da Superdotação já pode ser...,"""Curitiba"" - Google Notícias"
1011,Tarcísio foi na onda do Centrão e se deu mal c...,g1 > Política
4220,11° FEIRINHA DO SERVIDOR - Iniciativa da Prefe...,"""Boa Vista"" - Google Notícias"
3226,BH terá Tarifa Zero? Saiba quem são os vereado...,"""Belo Horizonte"" - Google Notícias"
2620,Atendentes da Sefin passam a usar fardamento p...,"""Vitória"" - Google Notícias"
2868,O Maranhão municipalista que estamos construin...,"""São Luís"" - Google Notícias"


--------------------------------------------------

Pergunta do Usuário: O que o prefeito de joão pessoa tem feito?

Resposta do Chatbot:
Com base nas notícias analisadas, a Justiça Eleitoral cassou os mandatos do prefeito e vice-prefeito de Soledade, no Cariri paraibano, por abuso de poder econômico na organização da Festa do Queijo em 2024, ano eleitoral. O município investiu R$ 621 mil na festa, mais de 3.000% acima do valor registrado em 2023, comprometendo a igualdade da disputa eleitoral. A magistrada considerou o aumento "exorbitante e injustificado". O prefeito afirmou respeitar a decisão, mas disse que ela causa "profunda estranheza", pois o Ministério Público havia se posicionado pela improcedência da ação. A defesa pretende recorrer da decisão, e o prefeito continua no exercício do mandato até o julgamento em instâncias superiores. Soledade estava sob decreto de emergência devido à estiagem, o que, segundo a magistrada, torna o gasto ainda mais relevante para o eleitorado.
